In [2]:
import scipy.io as sci
import numpy as np
import glob
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import stats
import pylab
import hddm

C:\Users\Edwin\miniconda3\envs\hddm\lib\site-packages\IPython\parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


In [28]:
#datapath = 'ExpData/*.mat'
#datafiles = np.array(glob.glob(datapath))
#datafiles
datafiles = ['ExpData\\axa_BEEPFLASH_1_28_10_0.mat', 'ExpData\\axt_FLASHBEEP_1_22_10_46.mat']

### Format Data

In [29]:
flashdata = np.empty((1,5))
beepdata = np.empty((1,5))
for i in np.arange(0, len(datafiles)):
    mat = sci.loadmat(datafiles[i])
    matf = np.insert(mat['mixtrF'], 0, int(i + 1), axis=1)
    accf = np.hstack((matf, mat['accMatF'], mat['resptimeF']))
    matb = np.insert(mat['mixtrB'], 0, int(i + 1), axis=1)
    accb = np.hstack((matb, mat['accMatB'], mat['resptimeB']))
    flashdata = np.vstack((flashdata, accf))
    beepdata = np.vstack((beepdata, accb))
    
flashdata = flashdata[1:]
beepdata = beepdata[1:]

flashframe = pd.DataFrame(flashdata, columns=['subj_idx', 'flashpres', 'beeppres', 'acc', 'rt'])
beepframe = pd.DataFrame(beepdata, columns=['subj_idx', 'flashpres', 'beeppres', 'acc', 'rt'])

congrf = flashframe.loc[flashframe['flashpres'] == flashframe['beeppres']]
unif = flashframe.loc[flashframe['beeppres'] == 0]
congrb = beepframe.loc[beepframe['flashpres'] == beepframe['beeppres']]
unib = beepframe.loc[beepframe['flashpres'] == 0]

def droprow(df, val, less=True):
    if(less):
        return(df.drop(df.index[df['rt'] < val].tolist(), axis=0))
    else:
        return(df.drop(df.index[df['rt'] > val].tolist(), axis=0))
    
def dropsubj(df, val):
    return(df.drop(df.index[df['subj_idx'] == val].tolist(), axis=0))

subject = 15

flashframe = droprow(flashframe, 0.05)
flashframe = dropsubj(flashframe, subject)
flashframe = droprow(flashframe, 10, False)
beepframe = droprow(beepframe, 0.05)
beepframe = dropsubj(beepframe, subject)
beepframe = droprow(beepframe, 10, False)

In [30]:
congrf = flashframe.loc[flashframe['flashpres'] == flashframe['beeppres']]
unif = flashframe.loc[flashframe['beeppres'] == 0]
congrb = beepframe.loc[beepframe['flashpres'] == beepframe['beeppres']]
unib = beepframe.loc[beepframe['flashpres'] == 0]

alldata = pd.concat([flashframe, beepframe])
alldata.columns = ['subj_idx', 'flashpres', 'beeppres', 'response', 'rt']
conditions = [(alldata['flashpres'] == 0) & (alldata['beeppres'] == 2), 
              (alldata['flashpres'] == 0) & (alldata['beeppres'] == 3),
              (alldata['flashpres'] == 2) & (alldata['beeppres'] == 0),
              (alldata['flashpres'] == 3) & (alldata['beeppres'] == 0),
              (alldata['flashpres'] == 2) & (alldata['beeppres'] == 2),
              (alldata['flashpres'] == 3) & (alldata['beeppres'] == 3),]
choices = ['A2', 'A3', 'V2', 'V3', 'A2V2', 'A3V3']
alldata['stimName'] = np.select(conditions, choices)

In [31]:
def formatmodeldata(df, conditions, choices):
    dframe = df.copy()
    dframe.columns = ['subj_idx', 'flashpres', 'beeppres', 'response', 'rt']
    dframe['condition'] = np.select(conditions, choices)
    return(dframe)

model_unif = formatmodeldata(unif, [(unif['flashpres'] == 2), (unif['flashpres'] == 3)], ['F2', 'F3'])
#model_unif = formatmodeldata(unif, [(unif['flashpres'] == 2), (unif['flashpres'] == 3)], ['easy', 'hard'])
model_unib = formatmodeldata(unib, [(unib['beeppres'] == 2), (unib['beeppres'] == 3)], ['B2', 'B3'])
model_congrf = formatmodeldata(congrf, [(congrf['flashpres'] == 2) & (congrf['beeppres'] == 2),
                                        (congrf['flashpres'] == 3) & (congrf['beeppres'] == 3)], ['F2B2', 'F3B3'])
model_congrb = formatmodeldata(congrb, [(congrb['flashpres'] == 2) & (congrb['beeppres'] == 2),
                                        (congrb['flashpres'] == 3) & (congrb['beeppres'] == 3)], ['F2B2', 'F3B3'])

### Match documentation formatting

In [32]:
model_unif = model_unif.drop(columns=['beeppres', 'flashpres'])
model_unif[1:5]
cols = model_unif.columns.tolist()
model_unif = model_unif[['rt', 'response', 'subj_idx', 'condition']]
model_unif.head(10)

,rt,response,subj_idx,condition
2,1.341606,1.0,1.0,F3
5,1.967426,1.0,1.0,F3
7,0.869609,1.0,1.0,F3
10,1.167405,1.0,1.0,F3
12,1.050508,1.0,1.0,F3
14,0.787502,1.0,1.0,F3
17,0.922216,1.0,1.0,F2
19,0.789067,1.0,1.0,F3
20,0.907375,1.0,1.0,F2
26,1.070062,1.0,1.0,F3


### Model and PPC

In [33]:
unif_acc = hddm.HDDM(model_unif, include=['a', 'v', 't', 'p_outlier'])
unif_acc.sample(7000, burn=500)

 [-----------------100%-----------------] 7000 of 7000 complete in 43.3 sec

In [34]:
ppc_data_pooled = hddm.utils.post_pred_gen(unif_acc, groupby=['condition'])
ppc_compare = hddm.utils.post_pred_stats(model_unif, ppc_data_pooled)

NotImplementedError: Supply a grouping so that at most 1 observed node codes for each group.

In [ ]:
print(ppc_compare)

In [16]:
data, params = hddm.generate.gen_rand_data(params={'F2':{'v': 1, 'a': 2, 't': .3}, 'F3':{'v': 2, 'a': 3, 't': 4}}, size = 200, subjs = 14)
data.head(10)

,rt,response,subj_idx,condition
0,1.904916,1.0,0,F2
1,1.629916,1.0,0,F2
2,0.560916,0.0,0,F2
3,1.173916,1.0,0,F2
4,1.145916,0.0,0,F2
5,1.555916,1.0,0,F2
6,1.237916,1.0,0,F2
7,0.822916,1.0,0,F2
8,0.809916,1.0,0,F2
9,0.815916,1.0,0,F2


In [18]:
m = hddm.HDDM(data)
m.sample(1000, burn=20)

 [-----------------100%-----------------] 1000 of 1000 complete in 121.9 sec

In [19]:
ppc_data_pooled = hddm.utils.post_pred_gen(m, groupby=['condition'])

NotImplementedError: Supply a grouping so that at most 1 observed node codes for each group.

In [22]:
#params#
num_samples = 5000
num_burn = 500

num_subs = 10
trials_per_v = 100
barrier = 2.5
ndt = 0.3
slope_A = 0.2
slope_B = 0.2
slope_C = 0.2
z_A = 0.5
z_B = 0.8
z_C = 0.3
params = {'A': {'a':barrier,'t':ndt,'v':slope_A,'z':z_A},
          'B': {'a':barrier,'t':ndt,'v':slope_B,'z':z_B},
          'C': {'a':barrier,'t':ndt,'v':slope_C,'z':z_C}
}
sim_data,sim_params = hddm.generate.gen_rand_data(params,size=trials_per_v,subjs=num_subs)

#model#
model_group = hddm.HDDM(sim_data,include='z',depends_on={'z':'condition'},informative=False)
model_group.sample(num_samples,burn=num_burn)

#plain model#
model_plain = hddm.HDDM(sim_data,include='z',informative=False)
model_plain.sample(num_samples,burn=num_burn)

##ppc##
num_ppc_samples = 50


 [-----------------100%-----------------] 5000 of 5000 complete in 683.8 sec

In [23]:
ppc_data_plain = hddm.utils.post_pred_gen(model_plain,samples = num_ppc_samples,groupby=['condition', 'subj_idx'])

 [------------------------------------------------------------330%-------------------------------------------------------------] 33 of 10 complete in 190.4 sec

ValueError: Length of names must match number of levels in MultiIndex.

In [ ]:
updated_name = ''
        name = updated_name.join(str(name))